# Suspicious Csc.exe Source File Folder
Detects a suspicious execution of csc.exe, which uses a source in a suspicious folder (e.g. AppData)

## Rule Content
```
- title: Suspicious Csc.exe Source File Folder
  id: dcaa3f04-70c3-427a-80b4-b870d73c94c4
  description: Detects a suspicious execution of csc.exe, which uses a source in a
    suspicious folder (e.g. AppData)
  status: experimental
  references:
  - https://securityboulevard.com/2019/08/agent-tesla-evading-edr-by-removing-api-hooks/
  - https://www.clearskysec.com/wp-content/uploads/2018/11/MuddyWater-Operations-in-Lebanon-and-Oman.pdf
  - https://app.any.run/tasks/c6993447-d1d8-414e-b856-675325e5aa09/
  - https://twitter.com/gN3mes1s/status/1206874118282448897
  author: Florian Roth
  date: 2019/08/24
  modified: 2019/12/17
  tags:
  - attack.defense_evasion
  - attack.t1500
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image: '*\csc.exe'
      CommandLine:
      - '*\AppData\\*'
      - '*\Windows\Temp\\*'
    filter:
      ParentImage:
      - C:\Program Files*
      - '*\sdiagnhost.exe'
      - '*\w3wp.exe'
    condition: selection and not filter
  falsepositives:
  - https://twitter.com/gN3mes1s/status/1206874118282448897
  - https://twitter.com/gabriele_pippi/status/1206907900268072962
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\csc.exe AND process_command_line.keyword:(*\\AppData\\* OR *\\Windows\\Temp\\*)) AND (NOT (process_parent_path.keyword:(C\:\\Program\ Files* OR *\\sdiagnhost.exe OR *\\w3wp.exe))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()